In [18]:
import pandas as pd
import geopandas as gpd
import os
import glob
import numpy as np
from dask.distributed import Client, LocalCluster
import multiprocessing as mp
import dask.dataframe as dd
import dask

In [53]:
il_csv=pd.read_csv('../../../Downloads/SIRNAK.csv')

In [54]:
il_csv.head(1)

,CityId,Field_Id,Il_Adi,Ilce_Adi,Il_adi_en,PlaceName,weather_stat_id,agri_stat_id,area,texture,...,sekerpancari_yield,SoyaFasulyesi_yield,Yerfistigi_yield,Yulaf_yield,Patates_yield,Misir_yield,Kanola_yield,Sogan_yield,Fasulye_yield,Domates_yield
0,73,5212060,ŞIRNAK,CİZRE,SIRNAK,Korucu,1228,59,3.399106,1,...,-13.126,-15.126,-15.126,-14.926,-36.226,-11.126,-20.926,-5.126,-25.126,-27.226


In [14]:
non_weather_yield=pd.read_csv('../../../Downloads/non_weather_yields_v1.csv',delimiter=';')
non_weather_yield=non_weather_yield[0:17]
non_weather_yield.loc[non_weather_yield['crop']=='Kanola (Kolza)','crop']='Kanola'
non_weather_yield.loc[non_weather_yield['crop']=='Soya Fasulyesi','crop']='SoyaFasulyesi'
non_weather_yield["crop_id"] = non_weather_yield["crop_id"].astype(int)
#non_weather_yield.iloc[0]

keys=sorted(non_weather_yield.crop_id.values)
values=sorted(non_weather_yield.crop.values)
crops_id_df=non_weather_yield[['crop','crop_id']]
crops_dict={}
for index,i in crops_id_df.iterrows():
    crop=i.values
    crops_dict[int(f'{crop[1]}')]=crop[0]


In [52]:
@dask.delayed
def irrigated_filter(row,crop_id):
    if row['nonirrigated']==False and row['irrigated']==False:
        return int(0)        
    elif row['nonirrigated']:
        return (float(non_weather_yield.loc[non_weather_yield['crop_id']==crop_id,'water_yloss'].values[0])*100)*(-1)
    else:
        return int(0) 
@dask.delayed
def slope_filter(row,crop_id):
    slope_th=float(non_weather_yield.loc[non_weather_yield['crop_id']==crop_id,'slope_treshold'].values[0])*100
    if row['slope_mean']>=slope_th:
        return (float(non_weather_yield.loc[non_weather_yield['crop_id']==crop_id,'slope_yloss'].values[0])*100)*(-1)
    else:
        return int(0)
@dask.delayed        
def ph_filter(row,crop_id):
    field_ph=float(row['phh2o_median'])/10
    ph_min=float(non_weather_yield.loc[non_weather_yield['crop_id']==crop_id,'ph_min'].values[0])
    ph_max=float(non_weather_yield.loc[non_weather_yield['crop_id']==crop_id,'ph_max'].values[0])
    if field_ph<ph_min:
        weight=(float(ph_min-field_ph))/float(0.1)
        return (float(non_weather_yield.loc[non_weather_yield['crop_id']==crop_id,'ph_low_yloss'].values[0])*weight)*(-1)*100
    elif field_ph>ph_max:
        weight=(float(field_ph-ph_max))/float(0.1)
        return (float(non_weather_yield.loc[non_weather_yield['crop_id']==crop_id,'ph_high_yloss'].values[0])*weight)*(-1)*100
    else:
        return int(0)
@dask.delayed    
def soc_filter(row,crop_id):
    field_soc=float(row['phh2o_median'])/10000
    soc_thr=float(non_weather_yield.loc[non_weather_yield['crop_id']==crop_id,'soc_treshold'].values[0])
    
    if field_soc<soc_thr:
        weight=(float(soc_thr-field_soc))/float(0.1)
        return (float(non_weather_yield.loc[non_weather_yield['crop_id']==crop_id,'soc_yloss'].values[0])*weight)*(-1)*100
    elif field_soc>soc_thr:
        weight=(float(field_soc-soc_thr))/float(0.1)
        return (float(non_weather_yield.loc[non_weather_yield['crop_id']==crop_id,'soc_ywin'].values[0])*weight)*100
    else:
        return int(0)
@dask.delayed   
def texture_filter(row,crop_id):
    good_text=non_weather_yield.loc[non_weather_yield['crop_id']==crop_id,'good_soil_texture'].values[0].split('-')
    bad_text=non_weather_yield.loc[non_weather_yield['crop_id']==crop_id,'bad_soil_texture'].values[0].split('-')
    texture_list=str(row['texture']).split('-')
    
    if set(good_text) & set(texture_list):
        return (float(non_weather_yield.loc[non_weather_yield['crop_id']==crop_id,'good_soil_texture_ywin'].values[0]))*100
    
    elif set(bad_text) & set(texture_list):
        return (float(non_weather_yield.loc[non_weather_yield['crop_id']==crop_id,'bad_soil_texture_ywin'].values[0]))*100
    else:
        return int(0)
@dask.delayed    
def runoff_filter(row,crop_id):
    row_runoff=row['runoff']
    if row_runoff=='0':
        return int(0) 
    return (float(non_weather_yield.loc[non_weather_yield['crop_id']==crop_id,str(row_runoff)].values[0])*100)

In [50]:
def filter_functions(row,crop_id):
    irrigated_filt=irrigated_filter(row,crop_id)
    slope_filt=slope_filter(row,crop_id)
    ph_filt=ph_filter(row,crop_id)
    soc_filt=soc_filter(row,crop_id)
    texture_filt=texture_filter(row,crop_id)
    runoff_filt=runoff_filter(row,crop_id)
    loss=irrigated_filt+slope_filt+ph_filt+soc_filt+texture_filt+runoff_filt
    results = dask.compute(loss)
    return results

In [55]:
cluster=LocalCluster(n_workers=int(1),
    processes=False,
    threads_per_worker=10,
    memory_limit='15GB',)
    

C:\Users\akif\.conda\envs\geo\lib\site-packages\distributed\node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 56714 instead
  http_address["port"], self.http_server.port


In [23]:
client=Client(cluster)
client

Client Scheduler: inproc://192.168.1.102/29452/1 Dashboard: http://192.168.1.102:8787/status,Cluster Workers: 1 Cores: 10 Memory: 15.00 GB


In [37]:
client.close()
cluster.close()

In [56]:
%%time
def yield_function(df):
    #df=df.persist()
    df['bugday']=df.apply(lambda x:filter_functions(row=x,crop_id=16),axis=1)
    return df    

il_csv=yield_function(il_csv)

Wall time: 8min 45s


In [47]:
il_csv.head()

,CityId,Field_Id,Il_Adi,Ilce_Adi,Il_adi_en,PlaceName,weather_stat_id,agri_stat_id,area,texture,...,SoyaFasulyesi_yield,Yerfistigi_yield,Yulaf_yield,Patates_yield,Misir_yield,Kanola_yield,Sogan_yield,Fasulye_yield,Domates_yield,bugday
0,73,5212060,ŞIRNAK,CİZRE,SIRNAK,Korucu,1228,59,3.399106,1,...,-15.126,-15.126,-14.926,-36.226,-11.126,-20.926,-5.126,-25.126,-27.226,"(0.074,)"
1,73,5212061,ŞIRNAK,CİZRE,SIRNAK,Korucu,1228,59,1.392393,1,...,-15.126,-15.126,-14.926,-36.226,-11.126,-20.926,-5.126,-25.126,-27.226,"(0.074,)"
2,73,5212062,ŞIRNAK,CİZRE,SIRNAK,Korucu,1228,59,1.597488,1,...,-25.126,-25.126,-14.926,-46.226,-21.126,-30.926,-15.126,-35.126,-37.226,"(0.074,)"
3,73,5212063,ŞIRNAK,CİZRE,SIRNAK,Korucu,1228,59,0.660651,1,...,-15.126,-15.126,-14.926,-36.226,-11.126,-20.926,-5.126,-25.126,-27.226,"(0.074,)"
4,73,5212064,ŞIRNAK,CİZRE,SIRNAK,Yavşan,1228,59,0.327918,1,...,-15.126,-15.126,-14.926,-36.226,-11.126,-20.926,-5.126,-25.126,-27.226,"(0.074,)"


In [ ]:
'''
Client
Scheduler: inproc://192.168.1.102/34544/9
Dashboard: http://192.168.1.102:8787/status
Cluster
Workers: 1
Cores: 10
Memory: 15.00 GB
Wall time: 6min 57s
'''

In [57]:
import pandas as pd
import time
from multiprocessing.dummy import Pool

pool_size = 5

pool = Pool(pool_size)


def add(a,b):
    sum = a+b
    time.sleep(2) #just to show that in reality my function takes times
    print("sum of %d and %d is %d" %(a, b, sum))

data = [[10,10],[9,12],[100,13]]
df = pd.DataFrame(data,columns=['col_1','col_2'])

start_time = time.time()
for ind in df.index:
     pool.apply_async(add, args=(df['col_1'][ind], df['col_2'][ind],))
pool.close()
pool.join()



print("--- %s seconds ---" % (time.time() - start_time))

sum of 10 and 10 is 20sum of 100 and 13 is 113sum of 9 and 12 is 21


--- 2.067957639694214 seconds ---
